In [1]:
import pandas as pd

In [112]:
results = pd.read_csv('results.csv', dtype={'season':str})

In [113]:
results.head()

,season,round,tie,winner,pk,agr,aet,result
0,2008,first,arsenal-milan,arsenal,False,False,False,arsenal (2-0) milan
1,2008,first,celtic-barcelona,barcelona,False,False,False,celtic (2-4) barcelona
2,2008,first,fenerbahce-sevilla,fenerbahce,True,False,True,"fenerbahce (5-5 aet) sevilla, fenerbahce won o..."
3,2008,first,liverpool-inter,liverpool,False,False,False,liverpool (3-0) inter
4,2008,first,lyon-united,united,False,False,False,lyon (1-2) united


In [114]:
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 8 columns):
season    162 non-null object
round     162 non-null object
tie       162 non-null object
winner    162 non-null object
pk        162 non-null bool
agr       162 non-null bool
aet       162 non-null bool
result    162 non-null object
dtypes: bool(3), object(5)
memory usage: 6.9+ KB


In [115]:
goles = pd.read_csv('goles - liga de campeones - Sheet1.tsv', sep='\t', dtype=str)

In [116]:
goles['season'].fillna(method='pad', inplace=True)
goles['round'].fillna(method='pad', inplace=True)
goles['tie'].fillna(method='pad', inplace=True)
goles['game'].fillna(method='pad', inplace=True)

In [117]:
goles.head()

,season,round,tie,game,minute,away,extra,shootout,note
0,2017,first,city-monaco,1,26,NaN,NaN,NaN,NaN
1,2017,first,city-monaco,1,32,a,NaN,NaN,NaN
2,2017,first,city-monaco,1,40,a,NaN,NaN,NaN
3,2017,first,city-monaco,1,58,NaN,NaN,NaN,NaN
4,2017,first,city-monaco,1,61,a,NaN,NaN,NaN


In [118]:
goles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 961 entries, 0 to 960
Data columns (total 9 columns):
season      961 non-null object
round       961 non-null object
tie         961 non-null object
game        961 non-null object
minute      936 non-null object
away        366 non-null object
extra       12 non-null object
shootout    7 non-null object
note        0 non-null object
dtypes: object(9)
memory usage: 67.6+ KB


In [119]:
goles[goles.minute.fillna('').str.contains('\+')].minute.unique()

array(['90+1', '90+5', '45+1', '90+6', '90+4', '90+3', '90+2', '45+2',
       '45+3', '90+8'], dtype=object)

In [120]:
goles.minute.unique()

array(['26', '32', '40', '58', '61', '71', '77', '82', '8', '29', '18',
       '49', '54', '24', '52', '57', '90+1', '48', '4', '85', '59', '11',
       '30', '53', '56', '63', '88', '20', '55', '68', '78', '80', '72',
       '74', '42', '17', '25', '86', nan, '3', '50', '90+5', '62', '73',
       '27', '28', '84', '19', '79', '35', '81', '47', '76', '105', '110',
       '112', '7', '22', '10', '12', '16', '33', '44', '69', '89', '60',
       '64', '39', '45+1', '67', '83', '65', '51', '43', '5', '108', 'pk',
       '90+6', '15', '2', '38', '36', '41', '31', '96', '114', '13', '70',
       '45', '34', '75', '87', '90+4', '14', '23', '90', '6', '9', '21',
       '37', '90+3', '66', '90+2', '1', '45+2', '45+3', '46', '98', '117',
       '90+8'], dtype=object)

In [121]:
def make_minutes(row):
    '''
    1-45: Game 1 first half (1-45)
    46: Game 1 first-half stoppage time
    47-91: Game 1 second-half (46-90)
    92: Game 1 second-half stoppage time
    
    93-137: Game 2 first half (1-45)
    138: Game 2 first-half stoppage time
    139-183: Game 2 second half (46-90)
    184: Game 2 second-half stoppage time
    
    185-199: Game 2 extra time first half (91-105)
    200: Game 2 extra time first half stoppage time
    201-215: Game 2 extra time second half (106-120)
    216: Game 2 extra time second half stoppage time
    
    217: Penalty shootout
    '''
    
    g = row.game
    m = row.minute
        
    if g == '1':
        if pd.isnull(m):
            return m
        if '45+' in m:
            return 46
        if '90+' in m:
            return 92
        if int(m) >= 1 and int(m) <= 45:
            return int(m)
        if int(m) >= 46 and int(m) <= 90:
            return int(m) + 1
        
    if g == '2':
        if pd.isnull(m):
            return m
        if '45+' in m:
            return 138
        if '90+' in m:
            return 184
        if '105+' in m:
            return 200
        if '120+' in m:
            return 216
        if 'pk' in m:
            return 217
        if int(m) >= 1 and int(m) <= 45:
            return int(m) + 92
        if int(m) >= 46 and int(m) <= 90:
            return int(m) + 93
        if int(m) >= 91 and int(m) <= 105:
            return int(m) + 94
        if int(m) >= 106 and int(m) <= 120:
            return int(m) + 95

In [122]:
goles['made_minute'] = goles.apply(make_minutes, axis=1)

In [123]:
ties = list(goles.groupby(['season','round','tie',]))

In [217]:
def create_matrix(df):
    result = df[['season','round','tie']].drop_duplicates().merge(results)
    et = (result['aet'].sum() > 0)
    made_min_max = 184 if not et else 217
    
    goals = df[['season','round','tie','made_minute','away']]
    goals = goals[goals.made_minute.notnull()]
    goals.away.fillna('h', inplace=True)
    
    all_minutes = goals.merge(
        pd.DataFrame({'made_minute':range(1,made_min_max+1)}),
        how='right'
    ).sort_values('made_minute')
    
    all_minutes['t1'] = all_minutes.tie.apply(lambda x: x.split('-')[0])
    all_minutes['t2'] = all_minutes.tie.apply(lambda x: x.split('-')[1])
    
#     all_minutes = all_minutes.merge(result[['season','round','tie','winner']], how='left')
#     all_minutes = all_minutes.drop(['season','round','tie',], axis=1)
#     all_minutes['winner'] = all_minutes['winner']\
#     .fillna(method='ffill').fillna(method='bfill').fillna(method='ffill').fillna(method='bfill')
    
    all_minutes['t1goal'] = (
        ((all_minutes['away'] == 'h') & (all_minutes['made_minute'] <= 92)) |
        ((all_minutes['away'] == 'a') & (all_minutes['made_minute'] > 92))
    )

    all_minutes['t2goal'] = (
        ((all_minutes['away'] == 'a') & (all_minutes['made_minute'] <= 92)) |
        ((all_minutes['away'] == 'h') & (all_minutes['made_minute'] > 92))
    )

    all_minutes['t1awaygoal'] = (all_minutes['t1goal'] & (all_minutes['away'] == 'a'))
    all_minutes['t2awaygoal'] = (all_minutes['t2goal'] & (all_minutes['away'] == 'a'))
    
#     all_minutes['t1goals'] = 
    
    return all_minutes

In [218]:
ties[50][1]

,season,round,tie,game,minute,away,extra,shootout,note,made_minute
565,2011,qtr,barcelona-shakhtar,1,2,NaN,NaN,NaN,NaN,2.0
566,2011,qtr,barcelona-shakhtar,1,34,NaN,NaN,NaN,NaN,34.0
567,2011,qtr,barcelona-shakhtar,1,53,NaN,NaN,NaN,NaN,54.0
568,2011,qtr,barcelona-shakhtar,1,61,NaN,NaN,NaN,NaN,62.0
569,2011,qtr,barcelona-shakhtar,1,86,NaN,NaN,NaN,NaN,87.0
570,2011,qtr,barcelona-shakhtar,1,60,a,NaN,NaN,NaN,61.0
571,2011,qtr,barcelona-shakhtar,2,43,a,NaN,NaN,NaN,135.0


In [219]:
allminutes = goles\
.groupby(['season','round','tie',])\
.apply(create_matrix)\
.reset_index()\
.drop('level_3', axis=1)

AttributeError: 'float' object has no attribute 'split'

In [220]:
allminutes

,season,round,tie,made_minute,away,t1,t2,winner,t1goal,t2goal,t1awaygoal,t2awaygoal
0,2008,first,arsenal-milan,1.0,NaN,arsenal,milan,arsenal,False,False,False,False
1,2008,first,arsenal-milan,2.0,NaN,arsenal,milan,arsenal,False,False,False,False
2,2008,first,arsenal-milan,3.0,NaN,arsenal,milan,arsenal,False,False,False,False
3,2008,first,arsenal-milan,4.0,NaN,arsenal,milan,arsenal,False,False,False,False
4,2008,first,arsenal-milan,5.0,NaN,arsenal,milan,arsenal,False,False,False,False
5,2008,first,arsenal-milan,6.0,NaN,arsenal,milan,arsenal,False,False,False,False
6,2008,first,arsenal-milan,7.0,NaN,arsenal,milan,arsenal,False,False,False,False
7,2008,first,arsenal-milan,8.0,NaN,arsenal,milan,arsenal,False,False,False,False
8,2008,first,arsenal-milan,9.0,NaN,arsenal,milan,arsenal,False,False,False,False
9,2008,first,arsenal-milan,10.0,NaN,arsenal,milan,arsenal,False,False,False,False
